# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

In [2]:
!pip install -U langchain langchain-openai langchain-cohere rank_bm25

We're also going to be leveraging [Qdrant's](https://qdrant.tech/documentation/frameworks/langchain/) (pronounced "Quadrant") VectorDB in "memory" mode (so we can leverage it locally in our colab environment).

In [3]:
!pip install -U qdrant-client

  Using cached qdrant_client-1.11.3-py3-none-any.whl.metadata (10 kB)
  Using cached grpcio-1.66.1-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached grpcio_tools-1.66.1-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached portalocker-2.10.1-py3-none-any.whl.metadata (8.5 kB)
  Using cached protobuf-5.28.2-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached setuptools-75.1.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached h2-4.1.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached hyperframe-6.0.1-py3-none-any.whl.metadata (2.7 kB)
  Using cached hpack-4.0.0-py3-none-any.whl.metadata (2.5 kB)
Using cached qdrant_client-1.11.3-py3-none-any.whl (258 kB)
Using cached grpcio-1.66.1-cp312-cp312-win_amd64.whl (4.3 MB)
Using cached grpcio_tools-1.66.1-cp312-cp312-win_amd64.whl (1.1 MB)
Using cached portalocker-2.10.1-py3-none-any.whl (18 kB)
Using cached h2-4.1.0-py3-none-any.whl (57 kB)
Using cached protobuf-5.28.2-cp310-abi3-win_amd64.whl (431 kB)
Using cached setupt

We'll also provide our OpenAI key, as well as our Cohere API key.

In [5]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

Enter your OpenAI API Key:··········


In [6]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

Cohere API Key:··········


## Task 2: Data Collection and Preparation

We'll be using some reviews from the 4 movies in the John Wick franchise today to explore the different retrieval strategies.

These were obtained from IMDB, and are available in the [AIM Data Repository](https://github.com/AI-Maker-Space/DataRepository).

### Data Collection

We can simply `wget` these from GitHub.

You could use any review data you wanted in this step - just be careful to make sure your metadata is aligned with your choice.

In [12]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
--2024-09-27 11:39:12--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv
Resolving raw.githubusercontent.com... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com|185.199.110.133|:443... connected.
OpenSSL: error:140770FC:SSL routines:SSL23_GET_SERVER_HELLO:unknown protocol
Unable to establish SSL connection.
SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
--2024-09-27 11:39:13--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv
Resolving raw.githubusercontent.com... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com|185.199.110.133|:443... connected.
OpenSSL: error:140770FC:SSL routines:SSL23_GET_SERVER_HELLO:unknown protocol
Unable to establish SSL connection.
SYSTEM_WGETRC = c:/progra~1/wget/e

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"],
      encoding="UTF-8"
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [2]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2024, 9, 24, 11, 47, 5, 638957)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [3]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(azure_deployment="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

In [30]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np  

cosine_similarity(np.array(embeddings.embed_query("mass-energy equivalence formula")).reshape(1, -1),
                  np.array(embeddings.embed_query("$$E = mc^2$$")).reshape(1, -1))

array([[0.65336683]])

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [39]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [40]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-3.5-turbo` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [41]:
from langchain_openai import AzureChatOpenAI

chat_model = AzureChatOpenAI(azure_deployment="gpt-35-turbo")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [42]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [43]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Overall, the reviews suggest that people generally liked John Wick. Many reviewers praised the film's action sequences, Keanu Reeves' performance as the titular character, and the film's stylish visuals. However, there were a few negative reviews as well. One reviewer did not understand the hype surrounding the film and found it to be a generic action thriller. Another reviewer felt that the magic of the first film was lost in the third installment."

In [44]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, one review has a rating of 10. Here is the URL to that review: '/review/rw4854296/?ref_=tt_urv'."

In [45]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In the original John Wick movie, an ex-hitman seeks revenge against the gangsters who killed his dog and took everything from him. He becomes the target of an army of bounty-hunting killers, and he unleashes a maelstrom of destruction against those who attempt to chase him. In the second movie, John Wick is forced back into the assassin world when he is called on to pay off an old debt. In the third movie, John Wick deals with the consequences of his actions in the previous film and continues to explore the world of assassination.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [46]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

We'll construct the same chain - only changing the retriever.

In [47]:
naive_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [48]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'It is difficult to determine whether people generally liked John Wick based on the given context as there are differing opinions expressed in the reviews provided.'

In [49]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, one review has a rating of 10. However, there are no URLs provided for the reviews in the given context.'

In [50]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'John Wick is a movie series known for its beautifully choreographed action scenes, emotional setup, and unique characters.'

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [51]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [52]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [53]:
contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the given context, it appears that people generally liked John Wick. The first two reviews gave it high ratings and praised its action sequences, characters, and overall entertainment value. However, the third review suggests that the magic may have been lost in the third installment.'

In [54]:
contextual_compression_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, one review has a rating of 10. Here is the URL to that review: '/review/rw4854296/?ref_=tt_urv'."

In [55]:
contextual_compression_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick 2, John Wick is asked by Santino D'Antonio to kill his sister Gianna D'Antonio in Rome so that Santino can sit on the High Table of the criminal organizations. After completing the task, Santino puts a seven-million dollar contract on John Wick, attracting professional killers from everywhere. John Wick promises to kill Santino, who is no longer protected by his marker. \n\nIn John Wick 1, an ex-hit-man seeks revenge after an arrogant Russian mob prince and hoodlums steal his car and kill his dog. He finds himself dragged into an impossible task as every killer in the business dreams of cornering the legendary Wick who now has an enormous price on his head. The legendary hitman will be forced to unearth his meticulously concealed identity and to carry out a relentless vendetta."

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [56]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [57]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [58]:
multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"People generally liked John Wick, with many reviewers praising its slick action sequences and Keanu Reeves' performance as the titular character. However, there were a few reviewers who were less impressed and found the film to be generic or lacking in plot. It is important to note that opinions on the film were not unanimous."

In [59]:
multi_query_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10 for John Wick 3. The URL for that review is '/review/rw4854296/?ref_=tt_urv'."

In [60]:
multi_query_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, an ex-hit-man comes out of retirement to seek revenge against the gangsters that killed his dog and took everything from him. He unleashes a carefully orchestrated maelstrom of destruction against those attempting to chase him, as he is the target of hit men. In John Wick 2, he is forced back into the assassin world when an Italian baddie calls in a favor and Wick has no choice but to accept. In John Wick 3, the film deals with the fallout of John's actions at the end of the previous film and sends him on an even bigger odyssey of violence."

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [61]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [64]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=embeddings, client=client
)

C:\Users\novikova\AppData\Local\Temp\ipykernel_55868\3287387143.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  parent_document_vectorstore = Qdrant(


Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [65]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [66]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [67]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [68]:
parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'It is unclear from the provided context whether people generally liked John Wick. There are mixed opinions expressed in the reviews.'

In [69]:
parent_document_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. The URL to that review is /review/rw4854296/?ref_=tt_urv.'

In [70]:
parent_document_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick 1, an ex-hitman seeks revenge when gangsters kill his dog and steal his car. In John Wick 2, John is called on to pay off an old debt by helping Ian McShane take over the Assassin's Guild by flying around to Italy, Canada, and Manhattan and killing what seems like hundreds of assassins."

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [71]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [72]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [73]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"The majority of the reviews suggest that people generally liked John Wick. There are numerous positive reviews that praise the film's action sequences, world-building, and Keanu Reeves' performance. However, there are also a few negative reviews that criticize the film for being too violent or lacking in plot. Overall, it seems that the film has a strong following among action fans."

In [74]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is one review with a rating of 10 for "John Wick 3". The URL to that review is \'/review/rw4854296/?ref_=tt_urv\'.'

In [75]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, an ex-hitman comes out of retirement to track down the gangsters that killed his dog and took everything from him. With the untimely death of his beloved wife still bitter in his mouth, he seeks vengeance. In John Wick 2, John Wick is called on to pay off an old debt by helping Ian McShane take over the Assassin's Guild by flying around to Italy, Canada and Manhattan and killing what seems like hundreds of assassins. In John Wick 3, John Wick deals with the consequences of his actions at the end of the previous film and goes on an even bigger odyssey of violence that continues to explore the world of assassination and deliver beautifully clean action sequences. As for John Wick 4, there are mixed reviews, with some finding it disappointing and others enjoying it."

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

In [76]:
!pip install -U langchain_experimental

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [77]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [78]:
semantic_documents = semantic_chunker.split_documents(documents)

Let's create a new vector store.

In [79]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

We'll use naive retrieval for this example.

In [80]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [81]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [82]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Overall, the reviews suggest that people generally liked John Wick. Many reviewers praised the action sequences, Keanu Reeves' performance, and the style of the film. However, there were a few negative reviews that criticized the lack of plot or character development."

In [83]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, one review has a rating of 10. The URL to that review is '/review/rw4854296/?ref_=tt_urv'."

In [84]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, an ex-hitman seeks revenge after gangsters kill his dog and steal his car, and he becomes the target of hitmen. In John Wick 2, John is forced back into the assassination world to pay off an old debt and is tasked with killing the sister of a mobster. In John Wick 3, John is on the run after being declared excommunicado and has to fight his way out of New York City. In John Wick 4, the plot is unclear from the reviews provided.'

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [86]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"],
      encoding="UTF-8"
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

## Creating a Golden Test Data Set

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from pydantic import BaseModel

generator_llm = AzureChatOpenAI(azure_deployment="gpt-4", temperature=0)
critic_llm = AzureChatOpenAI(azure_deployment="gpt-4", temperature=0)
embeddings = AzureOpenAIEmbeddings(azure_deployment="text-embedding-3-large")

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

num_qa_pairs = 20 # You can reduce the number of QA pairs to 5 if you're experiencing rate-limiting issues

testset = generator.generate_with_langchain_docs(eval_documents, num_qa_pairs, distributions, raise_exceptions=False)
testset.to_pandas()

embedding nodes:   0%|          | 0/264 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/20 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the priorities outlined in the White ...,[The White House (2022) Roadmap for Researcher...,The answer to given question is not present in...,simple,"[{'source': 'data/nist_ai.tex'}, {'source': 'd...",True
1,What type of information can provenance metada...,[Provenance metadata can include information a...,Provenance metadata can include information ab...,simple,[{'source': 'data/nist_ai.tex'}],True
2,"What processes are defined, assessed, and docu...",[MAP 3.4: Processes for operator and practitio...,Processes for operator and practitioner profic...,simple,[{'source': 'data/nist_ai.tex'}],True
3,What is the purpose of field testing in the ev...,[\item Al Red-teaming: A structured testing ex...,The purpose of field testing in the evaluation...,simple,[{'source': 'data/nist_ai.tex'}],True
4,What is the purpose of conducting periodic mon...,[\begin{center}\n\begin{tabular}{|c|c|c|}\n\hl...,The purpose of conducting periodic monitoring ...,simple,[{'source': 'data/nist_ai.tex'}],True
5,How should GAI acceptable use policies be upda...,[may rely on embedded GAI technologies; Addres...,GAI acceptable use policies should be updated ...,simple,[{'source': 'data/nist_ai.tex'}],True
6,What are the environmental impacts associated ...,[\item Confabulation: The production of confid...,Impacts due to high compute resource utilizati...,simple,[{'source': 'data/nist_ai.tex'}],True
7,How could model distillation or compression he...,[Trustworthy Al Characteristics: Accountable a...,Methods for creating smaller versions of train...,simple,[{'source': 'data/nist_ai.tex'}],True
8,What actions are suggested to address Human-AI...,[MAP 3.4: Processes for operator and practitio...,The suggested actions to address Human-AI Conf...,simple,[{'source': 'data/nist_ai.tex'}],True
9,How can structured public feedback be used to ...,[Measurement gaps can arise from mismatches be...,Structured public feedback can be used to eval...,simple,"[{'source': 'data/nist_ai.tex'}, {'source': 'd...",True


In [88]:
len(documents)

100

In [85]:
### YOUR CODE HERE

The Student's t-test is a statistical test used to compare the means of two groups. There are several key assumptions that must be met for the results of the t-test to be valid:

1. **Independence**: The observations within each group and between groups should be independent of each other. This means that the data points collected from one subject should not influence the data points collected from another subject.

2. **Normality**: The data in each group should be approximately normally distributed. This assumption is particularly important for small sample sizes. For larger sample sizes, the t-test is fairly robust to deviations from normality due to the Central Limit Theorem.

3. **Homogeneity of Variances (Homoskedasticity)**: The variances of the two groups should be equal. This assumption can be tested using Levene's test or an F-test. If the variances are not equal, a variation of the t-test known as Welch's t-test can be used.

4. **Scale of Measurement**: The dependent variable should be measured at the interval or ratio level, meaning that it should be continuous and have a meaningful zero point.

For a two-sample t-test specifically, these assumptions apply to both groups being compared. If any of these assumptions are violated, the results of the t-test may not be reliable, and alternative statistical methods may need to be considered.

In [ ]:
import numpy as np
from scipy import stats

# Example data for two groups
group1 = np.random.normal(loc=0, scale=1, size=30)
group2 = np.random.normal(loc=0, scale=1, size=30)

# Normality test using Shapiro-Wilk test
shapiro_test_group1 = stats.shapiro(group1)
shapiro_test_group2 = stats.shapiro(group2)

print("Shapiro-Wilk Test for Group 1:")
print(f"Statistic: {shapiro_test_group1.statistic}, p-value: {shapiro_test_group1.pvalue}")

print("Shapiro-Wilk Test for Group 2:")
print(f"Statistic: {shapiro_test_group2.statistic}, p-value: {shapiro_test_group2.pvalue}")

# Homogeneity of variances test using Levene's test
levene_test = stats.levene(group1, group2)

print("\nLevene's Test for Homogeneity of Variances:")
print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")

# Interpretation of p-values
alpha = 0.05

if shapiro_test_group1.pvalue > alpha:
    print("Group 1: Data is normally distributed (fail to reject H0)")
else:
    print("Group 1: Data is not normally distributed (reject H0)")

if shapiro_test_group2.pvalue > alpha:
    print("Group 2: Data is normally distributed (fail to reject H0)")
else:
    print("Group 2: Data is not normally distributed (reject H0)")

if levene_test.pvalue > alpha:
    print("Groups have equal variances (fail to reject H0)")
else:
    print("Groups do not have equal variances (reject H0)")